In [1]:
from ohlcv_data import OHLCVDataLoader

folder_path = "/Users/elyazidbenkhadra/Downloads/stock_market_data/sp500/csv"
combined_data = OHLCVDataLoader.load_all_data(folder_path, clean=True, years=20)

2025-04-17 20:22:53,693 - INFO - Preview of CSCO.csv:
         Date       Low  Open     Volume      High     Close  Adjusted Close
0  16-02-1990  0.073785   0.0  940636800  0.079861  0.077257        0.054862
1  20-02-1990  0.074653   0.0  151862400  0.079861  0.079861        0.056712
2  21-02-1990  0.075521   0.0   70531200  0.078993  0.078125        0.055479
3  22-02-1990  0.078993   0.0   45216000  0.081597  0.078993        0.056095
4  23-02-1990  0.078125   0.0   44697600  0.079861  0.078559        0.055787
2025-04-17 20:22:53,704 - INFO - Date parsing for CSCO.csv:
0   1990-02-16
1   1990-02-20
2   1990-02-21
3   1990-02-22
4   1990-02-23
Name: Date, dtype: datetime64[ns]
2025-04-17 20:22:53,708 - INFO - Filtered CSCO.csv from shape (8269, 7) to (4446, 7)
2025-04-17 20:22:53,709 - INFO - Loaded CSCO.csv with shape (4446, 6)
2025-04-17 20:22:53,704 - INFO - Date parsing for CSCO.csv:
0   1990-02-16
1   1990-02-20
2   1990-02-21
3   1990-02-22
4   1990-02-23
Name: Date, dtype: dateti

In [2]:
print(combined_data.head())

             CSCO_Low  CSCO_Open  CSCO_Volume  CSCO_High  CSCO_Close  \
Date                                                                   
2005-04-18  17.010000  17.170000     63901600  17.389999   17.020000   
2005-04-19  17.150000  17.200001     44281100  17.309999   17.180000   
2005-04-20  17.150000  17.430000     58411500  17.540001   17.190001   
2005-04-21  17.350000  17.389999     52371700  17.709999   17.680000   
2005-04-22  17.219999  17.500000     41057800  17.610001   17.430000   

            CSCO_Adjusted Close    UAL_Low  UAL_Open  UAL_Volume   UAL_High  \
Date                                                                          
2005-04-18            12.086415  33.700001     34.82   1940300.0  34.939999   
2005-04-19            12.200028  33.700001     34.82   1940300.0  34.939999   
2005-04-20            12.207131  33.700001     34.82   1940300.0  34.939999   
2005-04-21            12.555092  33.700001     34.82   1940300.0  34.939999   
2005-04-22           